<a href="https://colab.research.google.com/github/lailamt/timestamp_classifier/blob/main/Timestamp_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Libs and configs

In [2]:
!pip install num2words
# !pip install emoji
# !pip install demoji
!pip install unidecode
!pip install -U spacy
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=0984642739c07f21882892403c8fd06feb4812538b04c4e3b99ef26e05a00aa9
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 56.8 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.3
    Uninstalling spacy-3.5.3:
      Successfully uninstalled spacy-3.5.3
2023-07-03 17:15:52.832935: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the

In [3]:
import re
import os
import io
import string
import unidecode
# import emoji
# import demoji
import pandas as pd
from num2words import num2words
from collections import defaultdict

from wordcloud import WordCloud
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


###Paths

In [4]:
corpus_path = '/content/drive/MyDrive/Mestrado_UFBA/Tópicos em BD/corpus/'
prepro_path = '/content/drive/MyDrive/Mestrado_UFBA/Tópicos em BD/prepro/'
model_path = '/content/drive/MyDrive/Mestrado_UFBA/Tópicos em BD/modelos/'

###Read to dataframe

In [38]:
# Create an empty list to store dataframes
dfs = []

# Iterate over each file in the folder
for filename in os.listdir(corpus_path):
    file_path = os.path.join(corpus_path, filename)

    # Open the file and read its content
    with open(file_path, "r") as file:
        lines = file.readlines()

        # Extract the period from the first line and remove extra whitespaces and line breaks
        period = re.sub(r'\s+', ' ', lines[0].strip())

        # Join the remaining lines as a single text, remove extra whitespaces and line breaks
        text = re.sub(r'\s+', ' ', ' '.join(lines[1:]).strip())

        # Create a dataframe for the current file
        df = pd.DataFrame({"Period": [period], "Text": [text]})

        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all dataframes into a single dataframe
df = pd.concat(dfs, ignore_index=True)

In [39]:
df = df.sort_values(by=['Period']).reset_index(drop=True)

In [40]:
df['Text'][1]

"[ front ] [ end front ] [ title ] CHRONICA | DO MUITO ALTO; E MUITO ESCLARECIDO PRINCIPE | DOM DINIZ | SEXTO REY DE PORTUGAL, | COMPOSTA | POR RUY DE PINA, | Fidalgo da Caſa Real, e Chroniſta Môr do Reyno. | FIELMENTE COPIADA DO SEU ORIGINAL, | Que ſe conſerva no Archivo Real da Torre do Tombo. | OFFERECIDA | A' MAGESTADE SEMPRE AUGUSTA DELREY | D. JOAÕ O V. | NOSSO SENHOR. | | LISBOA OCCIDENTAL: | Na Officina F E R R E Y R I A N A. | M. DCC. XXIX. | Com todas as licenas neceſarias. | [ front ] [ end front ] CHRONICA | DELREY | D. DINIZ | SEXTO DE PORTUGAL; | [ prologue (author: )] [ end prologue ] | SENHOR | A OS Auguſtiſimos pés | de V. Mageſtade chega a minha obrigaçaõ a offerecerlhe | a Chronica do Senhor Rey D. Diniz ſeu duodecimo Avo. | | Eſta, Senhor, he a Hiſtoria de hũ dos mais glorioſos Prin- | cipes, que teve a Monarchia Portugueza, por~q te fez taõ | conhecido pela ſua prudencia, ~q dous grandes Reys o ele- | geraõ por arbitro, e Juiz das contendas, que lhe perturba- | vaõ

###Cleaning

In [41]:
stop_words = set(stopwords.words("portuguese"))

In [42]:
def clean(text):
    text = text.lower()
    text = re.sub(r'ſ', 's', str(text))
    text = re.sub(r'&', 'e', str(text))
    text = re.sub(r'-[ ]{2,}.*\n[ ]{2,}', '', str(text))
    text = re.sub(r'[ ]{2,}.*\n[ ]{2,}', ' ', str(text))
    text = re.sub(r'[àáâãäå]', 'a', str(text))
    text = re.sub(r'[èéêë]', 'e', str(text))
    text = re.sub(r'[ìíîï]', 'i', str(text))
    text = re.sub(r'[òóôõöø]', 'o', str(text))
    text = re.sub(r'[ùúûü]', 'u', str(text))

    text = re.sub(r'\[.*?\]', '', str(text))
    text = re.sub(r'\(.*?\)', '', str(text))

    pattern = re.compile(r'\d+')
    text = pattern.sub(lambda x: num2words(int(x.group()),
                                           lang='pt'), str(text))

    pattern = re.compile(r'\s{2,}')
    text = pattern.sub(' ', text).strip()

    text = re.sub(r'[^\w\' ]', '', str(text))
    text = re.sub(r'\n', '', str(text))

    text = [w for w in word_tokenize(text) if w not in stop_words]

    return text

In [43]:
df['Text'] = df['Text'].apply(clean)

In [44]:
df['Text'][1]

['chronica',
 'alto',
 'esclarecido',
 'principe',
 'dom',
 'diniz',
 'sexto',
 'rey',
 'portugal',
 'composta',
 'ruy',
 'pina',
 'fidalgo',
 'casa',
 'real',
 'chronista',
 'mor',
 'reyno',
 'fielmente',
 'copiada',
 'original',
 'conserva',
 'archivo',
 'real',
 'torre',
 'tombo',
 'offerecida',
 "'",
 'magestade',
 'sempre',
 'augusta',
 'delrey',
 'd',
 'joao',
 'v',
 'senhor',
 'lisboa',
 'occidental',
 'officina',
 'f',
 'r',
 'r',
 'y',
 'r',
 'i',
 'n',
 'm',
 'dcc',
 'xxix',
 'todas',
 'licenas',
 'necesarias',
 'chronica',
 'delrey',
 'd',
 'diniz',
 'sexto',
 'portugal',
 'senhor',
 'augustisimos',
 'pes',
 'v',
 'magestade',
 'chega',
 'obrigaçao',
 'offerecerlhe',
 'chronica',
 'senhor',
 'rey',
 'd',
 'diniz',
 'duodecimo',
 'avo',
 'senhor',
 'he',
 'historia',
 'hũ',
 'gloriosos',
 'prin',
 'cipes',
 'monarchia',
 'portugueza',
 'porq',
 'fez',
 'tao',
 'conhecido',
 'prudencia',
 'q',
 'dous',
 'grandes',
 'reys',
 'gerao',
 'arbitro',
 'juiz',
 'contendas',
 'perturb

####Salva arquivos preprocessados

In [45]:
def dataframe_to_txt_files(df, output_dir):

    for index, row in df.iterrows():
        file_name = os.path.join(output_dir, f"tb_prepro_{index}.txt")

        with open(file_name, 'w') as file:
            for column in df.columns:
                file.write(str(row[column]) + '\n')

In [46]:
dataframe_to_txt_files(df, prepro_path)

In [47]:
# Transforma o texto do sec. 14 em sec. 15
print(df['Period'][0])
df['Period'][0] = '1400-1499'
print(df['Period'][0])

1300-1399
1400-1499


In [48]:
# dropa a última linha (q é repetida)
df.drop(df.tail(1).index,inplace=True)
print(df)

       Period                                               Text
0   1400-1499  [chronica, delrey, d, ioam, i, boa, memoria, r...
1   1400-1499  [chronica, alto, esclarecido, principe, dom, d...
2   1400-1499  [comentario, arquivo, online, composto, comedi...
3   1400-1499  [auto, pastoril, portugues, representado, alto...
4   1400-1499  [chronica, delrey, daffonso, henriques, duarte...
..        ...                                                ...
77  1800-1899  [carta, cento, sete, arquivo, publico, estado,...
78  1800-1899  [revue, universelle, chegoume, aqui, boa, cida...
79  1800-1899  [senhoras, senhores, peça, vamos, ter, honra, ...
80  1800-1899  [carta, trezentos, onze, arquivo, barao, jerem...
81  1800-1899  [moises, visto, freud, gastao, pereira, silva,...

[82 rows x 2 columns]


###Creating subtexts

####subtexts

In [16]:
def chop_text_into_subtexts(text, subtext_length):
    subtexts = []
    num_subtexts = len(text) - subtext_length + 1
    for i in range(num_subtexts):
        subtext = text[i:i+subtext_length]
        subtexts.append(subtext)
    return subtexts

def create_subtexts_dataframe(df, subtext_length):
    subtexts = []
    centuries = []
    for i, row in df.iterrows():
        text = row['Text']
        century = row['Period']
        chopped_subtexts = chop_text_into_subtexts(text, subtext_length)
        subtexts.extend(chopped_subtexts)
        centuries.extend([century] * len(chopped_subtexts))
    subtexts_df = pd.DataFrame({'Text': subtexts, 'Period': centuries})
    return subtexts_df

# Assuming you have your original dataframe with 'Century' and 'Text' columns in 'df' variable
# Specify the desired subtext length
subtext_length = 100

# Create subtexts dataframe
subtexts_df = create_subtexts_dataframe(df, subtext_length)

# Print the first few rows of the subtexts dataframe
print(subtexts_df.head())

                                                Text     Period
0  [chronica, delrey, d, ioam, i, boa, memoria, r...  1400-1499
1  [delrey, d, ioam, i, boa, memoria, reys, portu...  1400-1499
2  [d, ioam, i, boa, memoria, reys, portugal, dec...  1400-1499
3  [ioam, i, boa, memoria, reys, portugal, decimo...  1400-1499
4  [i, boa, memoria, reys, portugal, decimo, prim...  1400-1499


In [17]:
print(subtexts_df['Period'].value_counts()['1400-1499'])
print(subtexts_df['Period'].value_counts()['1500-1599'])
print(subtexts_df['Period'].value_counts()['1600-1699'])
print(subtexts_df['Period'].value_counts()['1700-1799'])
print(subtexts_df['Period'].value_counts()['1800-1899'])

118715
321126
459933
361446
513230


####random subtexts

In [49]:
import pandas as pd
import random

def chop_text_into_subtexts(text, subtext_length):
    subtexts = []
    num_subtexts = len(text) - subtext_length + 1
    start_indices = random.sample(range(num_subtexts), num_subtexts)
    for start_index in start_indices:
        subtext = text[start_index:start_index+subtext_length]
        subtexts.append(subtext)
    return subtexts

def create_random_subtexts_dataframe(df, subtext_length, num_subtexts):
    subtexts = []
    centuries = []
    for i, row in df.iterrows():
        text = row['Text']
        period = row['Period']
        chopped_subtexts = chop_text_into_subtexts(text, subtext_length)
        random_subtexts = random.sample(chopped_subtexts, num_subtexts)
        subtexts.extend(random_subtexts)
        centuries.extend([period] * num_subtexts)
    subtexts_df = pd.DataFrame({'Text': subtexts, 'Period': centuries})
    return subtexts_df

# Assuming you have your original dataframe with 'Period' and 'Text' columns in 'df' variable
# Specify the desired subtext length and the number of random subtexts to generate per original text
subtext_length = 100
num_subtexts = 50

# Create random subtexts dataframe
random_subtexts_df = create_random_subtexts_dataframe(df, subtext_length, num_subtexts)

# Print the first few rows of the random subtexts dataframe
print(random_subtexts_df.head())

                                                Text     Period
0  [conselho, nao, verdadeiros, conselheiros, vẽ,...  1400-1499
1  [alfa, yates, querendo, tomar, mestre, senhor,...  1400-1499
2  [nao, prestarem, delleaco, tecendo, tomado, de...  1400-1499
3  [onde, vinha, elle, trazia, hũa, cota, vestida...  1400-1499
4  [corregedorm, outros, muytos, criados, rainha,...  1400-1499


In [50]:
print(random_subtexts_df['Period'].value_counts()['1400-1499'])
print(random_subtexts_df['Period'].value_counts()['1500-1599'])
print(random_subtexts_df['Period'].value_counts()['1600-1699'])
print(random_subtexts_df['Period'].value_counts()['1700-1799'])
print(random_subtexts_df['Period'].value_counts()['1800-1899'])

300
700
1100
800
1200


####join subtexts

In [60]:
def join_subtexts(text):
    return ' '.join(text)

In [61]:
random_subtexts_df['Text'] = random_subtexts_df['Text'].apply(join_subtexts)

In [22]:
random_subtexts_df.head()

,Text,Period
0,ate viesse vil la começou velar melhor ante na...,1400-1499
1,castella prendera iffante dom iohao conde dom ...,1400-1499
2,mo jaa dissemos parece ainda nao viera hora ca...,1400-1499
3,nuno aluerez partio pera casa ma neira viuer f...,1400-1499
4,parte alli erao asicarao escreuessemos iβo fra...,1400-1499


###Tf-idf / LinearSVC

####Train/test split

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
X = random_subtexts_df['Text']
y = random_subtexts_df['Period']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [53]:
print(X_train)
print(X_test)

4063    [bahia, nenhum, apareceu, tantas, vezes, tao, ...
2888    [necessario, dito, n, expresso, deitando, til,...
3534    [alferes, honorario, exercito, nomiado, decret...
815     [agente, especie, agregaçao, ve, panfilo, land...
207     [d, fernando, conde, trastamara, juntamente, r...
                              ...                        
1122    [diamantes, estao, casa, moeda, andem, pagar, ...
4060    [exacerbava, julgador, conceito, sobre, pedro,...
1346    [beltrao, afonso, mendes, burra, ja, jaz, po, ...
3454    [dito, havia, tido, educaçao, contraria, infor...
3582    [virgem, fez, iracema, guerreiro, branco, desv...
Name: Text, Length: 3280, dtype: object
553     [perdera, rosado, capitao, nau, francesa, mata...
3735    [tabelas, errava, medida, armaçoes, cemiterio,...
3244    [pasta, cinco, documento, contendo, dois, foli...
2813    [del, toboso, menos, porem, necessario, figura...
3928    [avos, custa, cinquentazero, reis, mes, nao, a...
                              ..

In [54]:
print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)

Training Data : (3280,)
Testing Data :  (820,)


####Vectorize

In [29]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

Accuracy: 0.9170731707317074
Classification Report:
              precision    recall  f1-score   support

   1400-1499       0.97      0.77      0.86        47
   1500-1599       0.93      0.92      0.92       150
   1600-1699       0.89      0.95      0.91       221
   1700-1799       0.91      0.84      0.87       151
   1800-1899       0.94      0.96      0.95       251

    accuracy                           0.92       820
   macro avg       0.93      0.89      0.90       820
weighted avg       0.92      0.92      0.92       820



In [ ]:
# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
# Initialize and train the LinearSVC model
svc_model = LinearSVC()
svc_model.fit(X_train_vectorized, y_train)

In [ ]:
# Predict the labels for test data
y_pred = svc_model.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

####Examples

In [30]:
svc_model.predict(vectorizer.transform(["Senhor, recebei minhas mais humildes reverências neste tempo de ilustração e progresso. Os ventos da mudança sopram sobre nossas terras, trazendo consigo novas ideias e conceitos. Os filósofos iluminados debatem ardorosamente sobre a natureza do homem e seus direitos inalienáveis. As cortes elegantes são palco de discussões intelectuais e sábios debates sobre política e governança. Enquanto isso, o comércio floresce e nossas colônias ultramarinas se expandem, trazendo riquezas e desafios para nossa nação. É um tempo de efervescência cultural, onde a arte, a música e a literatura encontram seu esplendor. Que a razão ilumine nosso caminho e nos guie rumo a um futuro próspero e justo."]))

array(['1800-1899'], dtype=object)

In [31]:
svc_model.predict(vectorizer.transform(["Senhor, a vossa graça seja bem-vinda nestas terras. De fato, os tempos são tumultuosos e o povo clama por justiça. A guerra alastra-se pelo reino, deixando um rastro de destruição e desespero por onde passa. Os senhores feudais lutam entre si, disputando o poder e a posse das terras. Enquanto isso, os camponeses trabalham arduamente, tentando sobreviver às agruras da vida. É uma época de incerteza e medo, onde a nobreza governa com mão de ferro e o povo sofre as consequências. Que Deus tenha misericórdia de nós e traga dias de paz e prosperidade novamente."]))

array(['1600-1699'], dtype=object)

In [32]:
#sec XV - Garcia de Resende
svc_model.predict(vectorizer.transform([
'''
Senhoras não hajais medo
não receeis fazer bem
tende o coração mui quedo
e vossas mercês verão cedo
quão grandes bens do bem vem.
Não torvem vosso sentido
as cousas qu'haveis ouvido
porqu'é lei de deos d'amor
bem, vertude nem primor
nunca jamais ser perdido.
Por verdes o galardão
que do amor recebeu
porque por ele morreu
nestas trovas saberão
o que ganhou ou perdeu.
Não perdeu senão a vida
que pudera ser perdida
sem na ninguém conhecer
e ganhou por bem querer
ser sua morte tão sentida.
''']))

array(['1500-1599'], dtype=object)

In [33]:
#sec XVI - Duarte Dias
svc_model.predict(vectorizer.transform([
'''
Logrando estou, senhora, um brando riso
Daquela doce boca e vista pura
Ua rara beleza, ua figura
Que me faz ver na terra o paraiso.
Logrando estou o delicado aviso,
A luz que torna dia a noite escura
O claro sol, a nova fermosura
Que abrasa o pensamento e perde o siso.
Logrando estou a graça peregrina,
A trança dos cabelos de ouro fino
Que em diferentes laços me arremata:
E quem logra tisouro tão divino
Claramente delira e desatina,
Se dele se apartar por ouro ou prata.
''']))

array(['1600-1699'], dtype=object)

In [34]:
#julho de 1843 - Gonçalves Dias
svc_model.predict(vectorizer.transform([
'''
Minha terra tem palmeiras
Onde canta o Sabiá,
As aves, que aqui gorjeiam,
Não gorjeiam como lá.
Nosso céu tem mais estrelas,
Nossas várzeas têm mais flores,
Nossos bosques têm mais vida,
Nossa vida mais amores.
Em cismar, sozinho, à noite,
Mais prazer encontro eu lá;
Minha terra tem palmeiras,
Onde canta o Sabiá.
Minha terra tem primores,
Que tais não encontro eu cá;
Em cismar – sozinho, à noite –
Mais prazer encontro eu lá;
Minha terra tem palmeiras,
Onde canta o Sabiá.
Não permita Deus que eu morra,
Sem que eu volte para lá;
Sem que desfrute os primores
Que não encontro por cá;
Sem qu’inda aviste as palmeiras,
Onde canta o Sabiá.
''']))

array(['1800-1899'], dtype=object)

In [35]:
# julho de 1928 - Carlos Drummond de Andrade
svc_model.predict(vectorizer.transform([
'''
No meio do caminho tinha uma pedra
tinha uma pedra no meio do caminho
tinha uma pedra
no meio do caminho tinha uma pedra.
Nunca me esquecerei desse acontecimento
na vida de minhas retinas tão fatigadas.
Nunca me esquecerei que no meio do caminho
tinha uma pedra
tinha uma pedra no meio do caminho
no meio do caminho tinha uma pedra.
''']))

array(['1500-1599'], dtype=object)

###Doc2vec / LinearSVC

In [57]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from gensim.models import Doc2Vec
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from sklearn import utils
import numpy as np

In [62]:
random_subtexts_df['Text']

0       conselho nao verdadeiros conselheiros vẽ elrey...
1       alfa yates querendo tomar mestre senhor nao co...
2       nao prestarem delleaco tecendo tomado dentro d...
3       onde vinha elle trazia hũa cota vestida ataavi...
4       corregedorm outros muytos criados rainha delre...
                              ...                        
4095    social democratico uniao democratica nacional ...
4096    recursos materiais ajudarnos mutuamente conseg...
4097    valor homem russo proteçao forças navais brita...
4098    tudo caso pensar vi nao nada imagine alguma co...
4099    chuva nao cessava so instante bares cafes nao ...
Name: Text, Length: 4100, dtype: object

In [64]:
# Prepare the tagged documents for Doc2Vec training
tagged_documents = [TaggedDocument(simple_preprocess(text), [i]) for i, text in enumerate(random_subtexts_df['Text'])]

In [65]:
# Initialize and train the Doc2Vec model
doc2vec_model = Doc2Vec(tagged_documents, vector_size=100, epochs=10)

In [67]:
# Vectorize the text using Doc2Vec
X_vectorized = np.array([doc2vec_model.infer_vector(simple_preprocess(text)) for text in random_subtexts_df['Text']])

In [68]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, random_subtexts_df['Period'], test_size=0.2, random_state=123)

In [69]:
# Initialize and train the LinearSVC model
svc_model = LinearSVC()
svc_model.fit(X_train, y_train)

LinearSVC()

In [70]:
# Predict the labels for test data
y_pred = svc_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.6268292682926829
Classification Report:
              precision    recall  f1-score   support

   1400-1499       0.61      0.57      0.59        47
   1500-1599       0.65      0.56      0.60       150
   1600-1699       0.59      0.57      0.58       221
   1700-1799       0.50      0.40      0.45       151
   1800-1899       0.70      0.86      0.77       251

    accuracy                           0.63       820
   macro avg       0.61      0.59      0.60       820
weighted avg       0.62      0.63      0.62       820



In [71]:
def vectorizer(text):
    return np.array([doc2vec_model.infer_vector(simple_preprocess(text))])

####Examples

In [72]:
svc_model.predict(vectorizer("Senhor, recebei minhas mais humildes reverências neste tempo de ilustração e progresso. Os ventos da mudança sopram sobre nossas terras, trazendo consigo novas ideias e conceitos. Os filósofos iluminados debatem ardorosamente sobre a natureza do homem e seus direitos inalienáveis. As cortes elegantes são palco de discussões intelectuais e sábios debates sobre política e governança. Enquanto isso, o comércio floresce e nossas colônias ultramarinas se expandem, trazendo riquezas e desafios para nossa nação. É um tempo de efervescência cultural, onde a arte, a música e a literatura encontram seu esplendor. Que a razão ilumine nosso caminho e nos guie rumo a um futuro próspero e justo."))

array(['1600-1699'], dtype=object)

In [73]:
svc_model.predict(vectorizer("Senhor, a vossa graça seja bem-vinda nestas terras. De fato, os tempos são tumultuosos e o povo clama por justiça. A guerra alastra-se pelo reino, deixando um rastro de destruição e desespero por onde passa. Os senhores feudais lutam entre si, disputando o poder e a posse das terras. Enquanto isso, os camponeses trabalham arduamente, tentando sobreviver às agruras da vida. É uma época de incerteza e medo, onde a nobreza governa com mão de ferro e o povo sofre as consequências. Que Deus tenha misericórdia de nós e traga dias de paz e prosperidade novamente."))

array(['1600-1699'], dtype=object)

In [75]:
#sec XV - Garcia de Resende
svc_model.predict(vectorizer(
'''
Senhoras não hajais medo
não receeis fazer bem
tende o coração mui quedo
e vossas mercês verão cedo
quão grandes bens do bem vem.
Não torvem vosso sentido
as cousas qu'haveis ouvido
porqu'é lei de deos d'amor
bem, vertude nem primor
nunca jamais ser perdido.
Por verdes o galardão
que do amor recebeu
porque por ele morreu
nestas trovas saberão
o que ganhou ou perdeu.
Não perdeu senão a vida
que pudera ser perdida
sem na ninguém conhecer
e ganhou por bem querer
ser sua morte tão sentida.
'''))

array(['1600-1699'], dtype=object)

In [76]:
#sec XVI - Duarte Dias
svc_model.predict(vectorizer(
'''
Logrando estou, senhora, um brando riso
Daquela doce boca e vista pura
Ua rara beleza, ua figura
Que me faz ver na terra o paraiso.
Logrando estou o delicado aviso,
A luz que torna dia a noite escura
O claro sol, a nova fermosura
Que abrasa o pensamento e perde o siso.
Logrando estou a graça peregrina,
A trança dos cabelos de ouro fino
Que em diferentes laços me arremata:
E quem logra tisouro tão divino
Claramente delira e desatina,
Se dele se apartar por ouro ou prata.
'''))

array(['1700-1799'], dtype=object)

In [78]:
#julho de 1843 - Gonçalves Dias
svc_model.predict(vectorizer(
'''
Minha terra tem palmeiras
Onde canta o Sabiá,
As aves, que aqui gorjeiam,
Não gorjeiam como lá.
Nosso céu tem mais estrelas,
Nossas várzeas têm mais flores,
Nossos bosques têm mais vida,
Nossa vida mais amores.
Em cismar, sozinho, à noite,
Mais prazer encontro eu lá;
Minha terra tem palmeiras,
Onde canta o Sabiá.
Minha terra tem primores,
Que tais não encontro eu cá;
Em cismar – sozinho, à noite –
Mais prazer encontro eu lá;
Minha terra tem palmeiras,
Onde canta o Sabiá.
Não permita Deus que eu morra,
Sem que eu volte para lá;
Sem que desfrute os primores
Que não encontro por cá;
Sem qu'inda aviste as palmeiras,
Onde canta o Sabiá.
'''))

array(['1600-1699'], dtype=object)

In [79]:
# julho de 1928 - Carlos Drummond de Andrade
svc_model.predict(vectorizer(
'''
No meio do caminho tinha uma pedra
tinha uma pedra no meio do caminho
tinha uma pedra
no meio do caminho tinha uma pedra.
Nunca me esquecerei desse acontecimento
na vida de minhas retinas tão fatigadas.
Nunca me esquecerei que no meio do caminho
tinha uma pedra
tinha uma pedra no meio do caminho
no meio do caminho tinha uma pedra.
'''))

array(['1500-1599'], dtype=object)

###salva

In [ ]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

# save the model to disk
filename = model_path + 'svc_model.sav'
joblib.dump(svc_model, filename)

['/content/drive/MyDrive/Mestrado_UFBA/Tópicos em BD/modelos/svc_model.sav']

In [ ]:
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, Y_test)
print(result)

#.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Assuming you have your text data and corresponding labels in X and y variables

# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Initialize and train the Multinomial Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_vectorized, y_train)

# Predict the labels for test data
y_pred = nb_model.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.9560794770722227
Classification Report:
              precision    recall  f1-score   support

   1300-1399       1.00      1.00      1.00      5864
   1400-1499       1.00      1.00      1.00     17640
   1500-1599       1.00      1.00      1.00     64426
   1600-1699       0.99      1.00      1.00     91929
   1700-1799       0.92      0.98      0.95     72277
   1800-1899       0.98      0.87      0.92    102668
   1900-1999       0.50      1.00      0.67      7464

    accuracy                           0.96    362268
   macro avg       0.91      0.98      0.93    362268
weighted avg       0.97      0.96      0.96    362268



In [ ]:
nb_model.predict(vectorizer.transform(["Senhor, recebei minhas mais humildes reverências neste tempo de ilustração e progresso. Os ventos da mudança sopram sobre nossas terras, trazendo consigo novas ideias e conceitos. Os filósofos iluminados debatem ardorosamente sobre a natureza do homem e seus direitos inalienáveis. As cortes elegantes são palco de discussões intelectuais e sábios debates sobre política e governança. Enquanto isso, o comércio floresce e nossas colônias ultramarinas se expandem, trazendo riquezas e desafios para nossa nação. É um tempo de efervescência cultural, onde a arte, a música e a literatura encontram seu esplendor. Que a razão ilumine nosso caminho e nos guie rumo a um futuro próspero e justo."]))

array(['1700-1799'], dtype='<U9')

In [ ]:
nb_model.predict(vectorizer.transform(["Senhor, a vossa graça seja bem-vinda nestas terras. De fato, os tempos são tumultuosos e o povo clama por justiça. A guerra alastra-se pelo reino, deixando um rastro de destruição e desespero por onde passa. Os senhores feudais lutam entre si, disputando o poder e a posse das terras. Enquanto isso, os camponeses trabalham arduamente, tentando sobreviver às agruras da vida. É uma época de incerteza e medo, onde a nobreza governa com mão de ferro e o povo sofre as consequências. Que Deus tenha misericórdia de nós e traga dias de paz e prosperidade novamente."]))

array(['1700-1799'], dtype='<U9')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Assuming you have your text data and corresponding labels in X and y variables

# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train_vectorized, y_train)

# Predict the labels for test data
y_pred = rf_model.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

In [ ]:
rf_model.predict(vectorizer.transform(["Senhor, recebei minhas mais humildes reverências neste tempo de ilustração e progresso. Os ventos da mudança sopram sobre nossas terras, trazendo consigo novas ideias e conceitos. Os filósofos iluminados debatem ardorosamente sobre a natureza do homem e seus direitos inalienáveis. As cortes elegantes são palco de discussões intelectuais e sábios debates sobre política e governança. Enquanto isso, o comércio floresce e nossas colônias ultramarinas se expandem, trazendo riquezas e desafios para nossa nação. É um tempo de efervescência cultural, onde a arte, a música e a literatura encontram seu esplendor. Que a razão ilumine nosso caminho e nos guie rumo a um futuro próspero e justo."]))

In [ ]:
rf_model.predict(vectorizer.transform(["Senhor, a vossa graça seja bem-vinda nestas terras. De fato, os tempos são tumultuosos e o povo clama por justiça. A guerra alastra-se pelo reino, deixando um rastro de destruição e desespero por onde passa. Os senhores feudais lutam entre si, disputando o poder e a posse das terras. Enquanto isso, os camponeses trabalham arduamente, tentando sobreviver às agruras da vida. É uma época de incerteza e medo, onde a nobreza governa com mão de ferro e o povo sofre as consequências. Que Deus tenha misericórdia de nós e traga dias de paz e prosperidade novamente."]))